## Web Scrap all trials
**Author:** Daniel Jaramillo

In [1]:
import pandas as pd
from pathlib import Path
from selenium import webdriver

from scrap_juicios import scrap_juicios

In [2]:
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# Define Paths
db = Path.cwd().parents[1]

In [4]:
# Load dependencias codes
db_depen = pd.read_csv(db/'data/raw/codigos_dependencias_satje.csv', header=None, names=['id_dependencia'])

# El Codigo de dependencia tiene 5 digitos. Lo convierto a string y anado un cero a los que tienen 4
db_depen['id_dependencia'] = db_depen['id_dependencia'].apply(lambda x: "0" + str(x) if len(str(x))==4 else str(x))
db_depen = db_depen.sort_values(by='id_dependencia').reset_index(drop=True)

1. Voy a tratar de obtener primero todos los juicios del Guayas

In [5]:
guayas = list(db_depen.loc[db_depen['id_dependencia'].apply(lambda x: x.startswith('09')), 'id_dependencia'])

La idea es tomar una dependencia, un ano, y probar todos los numeros posibles de procesos.

In [6]:
def juicios(driver, dep_judicial, year, n_attempt):

    # Create id_proceso
    id_proceso = (5-len(str(n_attempt)))*'0' + str(n_attempt)

    # Call function to webscrap
    result_list = scrap_juicios(driver, dep_judicial, year, id_proceso, delay=2)

    # Convert results to pandas
    result_df = pd.DataFrame()
    for instancia in result_list:
        result_df = pd.concat([result_df, pd.DataFrame(instancia, index=[0])], ignore_index=True)

    return result_df

In [7]:
# Define options for browser
options = webdriver.FirefoxOptions()
#options.headless = True # do not show browser window
options.page_load_strategy = 'none' # Dont wait page to be loaded

# Start Driver
gecko_path = 'C:/Users/DanielJaramillo/Documents/geckodriver.exe'
driver = webdriver.Firefox(executable_path=gecko_path, options=options)
url = 'http://consultas.funcionjudicial.gob.ec/informacionjudicial/public/informacion.jsf'

C:\Users\DanielJaramillo\AppData\Local\Temp\ipykernel_19180\3767969703.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=gecko_path, options=options)


In [8]:
# Define base for results
results = pd.DataFrame()
driver.get(url)

# Loop over years 
for year in range(2019, 2021):

    # Loop over possible trials
    for n_attempt in range(1, 99999+1):
        results_df = juicios(driver, '09100', str(year), n_attempt)
        
        if 'No existe este proceso' in results_df.causa[0]:
            break

        else:
            results = pd.concat([results, results_df], ignore_index=True)

driver.close()

ElementNotInteractableException: Message: Element <button id="form1:butLimpiar" class="ui-button ui-widget ui-state-default ui-corner-all ui-button-text-only" name="form1:butLimpiar" type="submit"> could not be scrolled into view
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
ElementNotInteractableError@chrome://remote/content/shared/webdriver/Errors.jsm:293:5
webdriverClickElement@chrome://remote/content/marionette/interaction.js:156:11
interaction.clickElement@chrome://remote/content/marionette/interaction.js:125:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:203:24
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:91:31


In [12]:
# Save results
results.to_excel(db/f'data/raw/resumen_09100.xls', index=False)